# Extração de Dados com Selenium

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def slow_scroll():
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.END)
    time.sleep(5)

url = 'https://www.metacritic.com/game/prince-of-persia-the-lost-crown/user-reviews/?platform=playstation-5'
option = Options()
option.headless = False
driver = webdriver.Firefox(options=option)
driver.get(url)

Problem reading geckodriver versions: error sending request for url (https://raw.githubusercontent.com/SeleniumHQ/selenium/trunk/common/geckodriver/geckodriver-support.json). Using latest geckodriver version
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [2]:
# Lista para armazenar os produtos de todas as páginas
reviews = []

# Loop para percorrer até o final
previous_height = driver.execute_script('return document.body.scrollHeight')

In [3]:
# Loop para percorrer até o final
while True:
    slow_scroll()
    new_height = driver.execute_script('return document.body.scrollHeight')
    
    # Se a altura anterior for igual à nova altura, terminamos de rolar
    if new_height == previous_height:
        break
    
    previous_height = new_height

    div_main = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[2]/div[1]/section/div[6]')
    html_content = div_main.get_attribute('outerHTML')
    soup = BeautifulSoup(html_content, 'html.parser')
    comentarios = soup.find_all('div', class_='c-siteReview g-bg-gray10 u-grid g-outer-spacing-bottom-large')
    reviews.extend(comentarios)

driver.quit()

In [4]:
dados_reviews = []

In [5]:
# Coletar texto e nome dos comentários
for review in reviews:
    texto_container = review.find('div', class_='c-siteReview_main g-inner-spacing-medium')
    # nome_container = review.find('a', class_='c-siteReviewHeader_username g-text-bold g-color-gray90 ')

    if texto_container:
        texto = texto_container.get_text(strip=True)
        
        # Adicionar os dados ao array
        dados_reviews.append({'Comentario': texto})

df = pd.DataFrame(dados_reviews)

In [6]:
df.to_csv("npl_princeofpersia_raw.csv")